In [18]:
from env import MONGO_CLOUD

import pymongo
import pandas as pd
import pytz
import time
import matplotlib.pyplot as plt
from matplotlib import pyplot
import datetime
import plotly.express as px
import os
import glob
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import interact, interact_manual

import plotly.express as px
import matplotlib.pyplot as plt 

In [19]:
def Create_3D(Value_Frame, Value_Frame_Z1, columns, Time, Color
                 , idx, roated=90, save=True,n_row=8, n_col=4):

    """
    Create a 3D surface plot for visualizing two sets of values as surfaces.

    Parameters:
    -----------
    Value_Frame : 2D array-like
        The primary data matrix used to plot the first surface.
    Value_Frame_Z1 : 2D array-like
        The secondary data matrix used to plot the second surface.
    columns : list
        Not used in the function.
    Time : datetime or string
        A timestamp used in the plot title.
    Color : bool
        If True, use 'coolwarm' colormap for both surfaces. 
        If False, use 'Greens' for the first surface and 'YlOrBr' for the second.
    idx : int or str
        The index or identifier used for naming the saved plot file.
    roated : int, optional (default=90)
        The azimuthal rotation angle of the plot.
    save : bool, optional (default=True)
        If True, save the plot as a PNG file in the '3D_Plots' directory with the filename containing `idx`.
    n_row : int, optional (default=8) - set the number of rows in the plot.
    n_col : int, optional (default=4) - set the number of columns in the plot.
    Returns:
    --------
    None
        The function generates and saves or displays a 3D surface plot.

    Notes:
    ------
    - Colormap is selected based on the `Color` parameter.
    - Plots two 3D surfaces using data matrices `Value_Frame` and `Value_Frame_Z1`.
    - Adjusts the aspect ratio to match real-world dimensions for better visualization.
    - Saves the plot to the '3D_Plots' directory if `save=True`. 
      The saved filename format is '{idx}_3D_plot.png'.
    
    Example:
    --------
    Create_graph(Value_Frame=df1, 
                 Value_Frame_Z1=df2, 
                 columns=some_columns, 
                 Time="2024-09-12 12:00:00", 
                 Color=True, 
                 idx=1, 
                 roated=45, 
                 save=True)
    """
    if Color: 
        # if color is True, use 'coolwarm' colormap for both surfaces
        # as it only one plot_surface function is used
        color_map = ['coolwarm', 'coolwarm']
    else:
        # if color is False, use 'Greens' for the first surface and 'YlOrBr' for the second
        # as two plot_surface functions are used
        color_map = ["Greens", "YlOrBr"]
        
    x = [i for i in range(1, n_row + 1)][::-1] # set the x-axis values for the plot
    y = [i for i in range(1, n_col + 1)] # set the y-axis values for the plot
    labels = [chr(num+96) for num in y]  # set the labels for the y-axis using ASCII values (1,2,3,4) -> (a,b,c,d)
    
    x, y = np.meshgrid(x, y)
    valueX = [i for i in range(1, n_row + 1)][::-1]
    valueY = [i for i in range(1, n_col + 1)]
    Z_0 = Value_Frame
    Z_1 = Value_Frame_Z1

    fig = plt.figure(figsize=(18, 6))  # set figure size
    ax = fig.add_subplot(111, projection='3d')  # set the figure to be in 3D

    ax1 = ax.plot_surface(x, y, Z_0, cmap=f"{color_map[0]}",
                          linewidth=.8, label='X=0', facecolor="blue",
                          edgecolor="black")  # Add title and labels to plot.
    ax1._facecolors2d = ax1._facecolor3d
    ax1._edgecolors2d = ax1._edgecolor3d

    #############################################
    ax2 = ax.plot_surface(x, y, Value_Frame_Z1, cmap=f"{color_map[1]}",
                          linewidth=.8, label='X=0', facecolor="blue",
                          edgecolor="black")  # Add title and labels to plot
    ax1._facecolors2d = ax1._facecolor3d
    ax1._edgecolors2d = ax1._edgecolor3d
    
    Grap_Title = str(Time).split("+")[0]
    plt.title(f"{Grap_Title}", fontsize=16)
    plt.yticks(valueY, labels)
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_xlim(ax.get_xlim())
    
    # Apply the box aspect ratio to match 6.4 meters by 2.4 meters
    ax.set_box_aspect([2.67, 1.5, 1.2])  # Custom aspect ratio to fit real-world dimensions (x, y, z)

    plt.margins(0.2)
    plt.subplots_adjust(bottom=0.15)
    ax.view_init(azim=-roated)
    # Increase tick size
    ax.tick_params(axis='both', which='major', labelsize=12)

 

In [28]:
import matplotlib.pyplot as plt
import numpy as np

def Create_3D(Value_Frame, Value_Frame_Z1, columns, Time, Color, idx, roated=90, save=True, n_row=8, n_col=4):
    if Color: 
        color_map = ['coolwarm', 'coolwarm']
    else:
        color_map = ["Greens", "YlOrBr"]
        
    # Create meshgrid
    x = [i for i in range(1, n_row + 1)][::-1]
    y = [i for i in range(1, n_col + 1)]
    x, y = np.meshgrid(x, y)
    
    # Create figure and axis
    fig = plt.figure(figsize=(18, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Plot first surface
    ax1 = ax.plot_surface(x, y, Value_Frame, cmap=f"{color_map[0]}", linewidth=.8, edgecolor="black")
    
    # Plot second surface only if Value_Frame_Z1 is different from Value_Frame
    if not np.array_equal(Value_Frame, Value_Frame_Z1):
        ax2 = ax.plot_surface(x, y, Value_Frame_Z1, cmap=f"{color_map[1]}", linewidth=.8, edgecolor="black")
    
    # Set plot title and other properties
    Grap_Title = str(Time).split("+")[0]
    plt.title(f"{Grap_Title}", fontsize=16)
    ax.set_box_aspect([2.67, 1.5, 1.2])  # Custom aspect ratio to fit real-world dimensions
    ax.view_init(azim=-roated)

    # Save or show the plot
    if save:
        plt.savefig(f"3D_Plots/{idx}_3D_plot.png")
    else:
        plt.show()

    # Explicitly close the plot after displaying or saving
    plt.close(fig)


In [29]:
## reading the file with the DATA
sample_df = pd.read_csv("Sample_Data.csv") 
# pivot the temperatute column for sample data
# Aggregate by taking the mean for duplicate 'TimeStamp' and 'Name'
sample_df_aggregated = sample_df.groupby(['TimeStamp', 'Name']).agg({'temperature': 'mean'}).reset_index()
# pivot the temperatute column for sample data
sample_df_aggregated = sample_df.groupby(['TimeStamp', 'Name']).agg({'temperature': 'mean'}).reset_index()
# Filtering the aggregated DataFrame to extract temperature columns for Z0 and Z1
temp_Z0_aggregated = sample_df_aggregated[sample_df_aggregated['Name'].str.contains('Z0')]
temp_Z1_aggregated = sample_df_aggregated[sample_df_aggregated['Name'].str.contains('Z1')]
# Pivoting the dataframes
pivot_temp_Z0_aggregated = temp_Z0_aggregated.pivot(index='TimeStamp', columns='Name', values='temperature')
pivot_temp_Z1_aggregated = temp_Z1_aggregated.pivot(index='TimeStamp', columns='Name', values='temperature')
## conact the two dataframes
df_concated = pd.concat([pivot_temp_Z0_aggregated, pivot_temp_Z1_aggregated], axis=1)
# sort the index
df_concated.sort_index(axis=1,inplace=True)

In [30]:
import matplotlib.pyplot as plt

@interact(Height=["Z0", "Z1", "Both", "Delta"], index=(0, len(df_concated.index) - 2), roated=(0, 360, 1), continuous_update=False)
def show_graph(Height="Z0", index=0, roated=216):
    plt.close('all')  # Close all previous plots to prevent duplication
    
    if Height in ["Z0", "Z1"]:
        df = df_concated[df_concated.filter(like=Height).columns]
        Value_Frame = df.iloc[index].values.reshape(8, 4)[::-1].transpose()
        Time = df.index[index]
        Create_3D(Value_Frame, Value_Frame, "Value", Time, True, index, roated=-roated, save=False)
        
    elif Height == "Both":
        df_0 = df_concated[df_concated.filter(like="Z0").columns]
        df_1 = df_concated[df_concated.filter(like="Z1").columns]

        Value_Frame_Z0 = df_0.iloc[index].values.reshape(8, 4)[::-1].transpose()
        Value_Frame_Z1 = df_1.iloc[index].values.reshape(8, 4)[::-1].transpose()
        Time = df_0.index[index]
        Create_3D(Value_Frame_Z0, Value_Frame_Z1, "Value", Time, False, index, roated=-roated, save=False)
        
    elif Height == "Delta":
        df_0 = df_concated[df_concated.filter(like="Z0").columns]
        df_1 = df_concated[df_concated.filter(like="Z1").columns]
    
        Value_Frame_Z0 = df_0.iloc[index].values.reshape(8, 4)[::-1].transpose()
        Value_Frame_Z1 = df_1.iloc[index].values.reshape(8, 4)[::-1].transpose()
        
        Value_Frame_Delta = Value_Frame_Z1 - Value_Frame_Z0
        Time = df_0.index[index]
        Create_3D(Value_Frame_Delta, Value_Frame_Delta, "Value", Time, True, index, roated=-roated, save=False)



interactive(children=(Dropdown(description='Height', options=('Z0', 'Z1', 'Both', 'Delta'), value='Z0'), IntSl…